In [1]:
%matplotlib inline
import pandas as pd
import os
import re
import shutil
import numpy as np
import warnings

Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 461 ('figure.figsize   : 5, 3   ## figure size in inches')
Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 462 ('figure.dpi       : 100        ## figure dots per inch')


In [20]:
df = pd.read_csv('../students.csv')

In [21]:
df

,family name,name
0,Dahlén,Isak
1,Dahlqvist,Petrus
2,Danielsson,Isac
3,Durkin,Kirill
4,Dynevik,Philip
5,Edsbagge,Carl
6,Egeland,Karl
7,Ekdahl,Jonathan
8,El Masri,Mohammed Ali
9,Elm Jonsson,Benjamin


In [22]:
regexp=re.compile(r'^ *(.*)')
def clean_names(name):
    result=regexp.search(name)
    return result.group(1)

In [23]:
df['name']=df['name'].apply(func=clean_names)

In [24]:
def full_name(row):
    return '%s %s' % (row['name'],row['family name'])

In [25]:
df.index = df.apply(func=full_name, axis=1)
assert df.index.is_unique

In [26]:
def create_folder(row,root_path='students/assignments'):
    
    family_name = row['family name']
    name = row['name']
    dir_name = '%s_%s' % (family_name,name)
    path = os.path.abspath(os.path.join(root_path,dir_name))
    if not os.path.exists(path):
        os.mkdir(path)
    
    return path
    

## Create folders:

In [27]:
root_path='students/assignments'

if not os.path.exists(root_path):
    os.mkdir(root_path)

df['dir_path'] = df.apply(func=create_folder, axis=1, root_path=root_path)

## Rename Canvas data

In [28]:
# trying to match person with files like:
#bergströmolssonamanda_48073_750610_svenskPunktlighetsStatistik.txt

s = 'bergströmolssonamanda_48073_750610_svenskPunktlighetsStatistik.txt'

In [29]:
parts = s.split('_')
id_string=parts[0]
file_name = parts[-1]
id_string

'bergströmolssonamanda'

In [30]:
df_files = pd.DataFrame()
directory_name=r'students/submissions'
for file_name_raw in os.listdir(directory_name):
    s_=pd.Series(name=file_name_raw)
    
    parts = file_name_raw.split('_')
    s_['id_string']=parts[0]
    s_['file_name'] = parts[-1]
    s_['file_path_raw'] = os.path.join(directory_name,file_name_raw)
    df_files=df_files.append(s_)

c:\dev\tme136-programming-and-algorithms\venv\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


In [31]:
df_files.head()

,file_name,file_path_raw,id_string
abadaemanuel_20815_787768_MagicK1.m,MagicK1.m,students/submissions\abadaemanuel_20815_787768...,abadaemanuel
abadaemanuel_20815_787769_readStockData.m,readStockData.m,students/submissions\abadaemanuel_20815_787769...,abadaemanuel
abadaemanuel_20815_787770_simulateStrategy.m,simulateStrategy.m,students/submissions\abadaemanuel_20815_787770...,abadaemanuel
abadaemanuel_20815_787771_printAndPlotResult.m,printAndPlotResult.m,students/submissions\abadaemanuel_20815_787771...,abadaemanuel
abdadeler_LATE_34710_807787_MagicK1-1.m,MagicK1-1.m,students/submissions\abdadeler_LATE_34710_8077...,abdadeler


In [34]:
def find_id(row):

    family_names=row['family name'].split(' ')
    names=row['name'].split(' ')
    
    df_search=df_files.copy()
    
    searches_name = [name.lower() for name in names]
    searches_family = [family_name.lower() for family_name in family_names]
    searches = searches_name + searches_family
    
    for search in searches:
    
        mask = df_files['id_string'].str.contains(search)
        df_search=df_search.loc[mask]
    
    if len(df_search['id_string'].unique())==0:
        id_string=None  # No files supplied for this student
    elif len(df_search['id_string'].unique())==1:
        id_string=df_search.iloc[0]['id_string']
    else:
        mask2 = df_search['id_string'].duplicated()
        raise ValueError('Duplicates:%s' % df_search.loc[mask2])
        
    return id_string

In [35]:
df['id_string'] = df.apply(func=find_id, axis=1)

## Copy and rename files

In [38]:
def copy_rename(row):
    id_string=row['id_string']
    mask=df_files['id_string']==id_string
    df_ = df_files.loc[mask]
    for index, sub_row in df_.iterrows():
        destination = os.path.join(row['dir_path'],sub_row['file_name'])
        if not os.path.exists(destination):
            shutil.copyfile(sub_row['file_path_raw'], destination)
    

In [39]:
df.apply(func=copy_rename, axis=1);

In [40]:
def make_clickable(val):
    s=r'file:///' + val
    return '<a href="{}">{}</a>'.format(s,"path")

In [41]:
def show(df):
    return df.style.format({'dir_path': make_clickable})

## Check files exist:

In [42]:
correct_code_src_dir=r'matlab'
src_files = [file for file in os.listdir(correct_code_src_dir) if os.path.splitext(file)[-1] == '.m']
assert len(src_files) > 0
    

def has_files(row, src_files):    
    
    for src_file in src_files:
        path=os.path.join(row['dir_path'],src_file)
        if not os.path.exists(path):
            return False
        
    return True

In [45]:
src_files

['MagicK1.m', 'printAndPlotResult.m', 'readStockData.m', 'simulateStrategy.m']

In [58]:
df['has_files'] = df.apply(func=has_files, src_files=src_files, axis=1)

In [59]:
mask=~df['has_files']
show(df.loc[mask])

,family name,name,dir_path,id_string,has_files
Johan Erlandsson,Erlandsson,Johan,path,None,False


In [60]:
def repair_file_names(row, src_files):
    
    for src_file in src_files:
        src_file_name, src_file_ext = os.path.splitext(src_file)
        for file in os.listdir(row['dir_path']):
            file_name, file_ext = os.path.splitext(file)
            if ((src_file_ext==file_ext) and
                (src_file_name != file_name) and
                (src_file_name in file_name)):
                src = os.path.join(row['dir_path'], file)
                dst = os.path.join(row['dir_path'], src_file)
                shutil.copyfile(src,dst)

In [61]:
df.apply(func=repair_file_names, axis=1, src_files=src_files)

df['has_files'] = df.apply(func=has_files, src_files=src_files, axis=1)
mask=~df['has_files']
show(df.loc[mask])

,family name,name,dir_path,id_string,has_files
Johan Erlandsson,Erlandsson,Johan,path,None,False


In [62]:
show(df)

,family name,name,dir_path,id_string,has_files
Isak Dahlén,Dahlén,Isak,path,dahlénisak,True
Petrus Dahlqvist,Dahlqvist,Petrus,path,dahlqvistpetrus,True
Isac Danielsson,Danielsson,Isac,path,danielssonisac,True
Kirill Durkin,Durkin,Kirill,path,durkinkirill,True
Philip Dynevik,Dynevik,Philip,path,dynevikphilip,True
Carl Edsbagge,Edsbagge,Carl,path,edsbaggecarl,True
Karl Egeland,Egeland,Karl,path,egelandkarl,True
Jonathan Ekdahl,Ekdahl,Jonathan,path,ekdahljonathan,True
Mohammed Ali El Masri,El Masri,Mohammed Ali,path,elmasrimohammedali,True
Benjamin Elm Jonsson,Elm Jonsson,Benjamin,path,elmjonssonbenjamin,True


## Running the scripts to get the times

In [147]:
def matlab_to_data_frame(keys,r):
    data={key:np.array(r[key])[0] for key in keys if key in r}
    df_ = pd.DataFrame(data)
    return df_

In [148]:
from contextlib import contextmanager

@contextmanager
def run_matlab():

    try:
        yield matlab.engine.start_matlab()
        
    except:
        # Do something here...
        eng.quit()
        raise
    else:
        eng.quit()

def copy_extra_files(project_path, extra_file_paths):
    for extra_file_path in extra_file_paths:
            extra_file = os.path.split(extra_file_path)[-1]
            new_path = os.path.join(project_path,extra_file)
            if not os.path.exists(new_path):
                shutil.copyfile(extra_file_path, new_path)
 
def remove_extra_files(project_path, extra_file_paths):
    
    for extra_file_path in extra_file_paths:
            extra_file = os.path.split(extra_file_path)[-1]
            new_path = os.path.join(project_path,extra_file)
            if os.path.exists(new_path):
                os.remove(new_path)

@contextmanager
def evaluate_path(project_path, eng, extra_file_paths=[], clean=True):
    old_path = eng.pwd()
    try:
        
        # Copy extra files that are needed:
        copy_extra_files(project_path=project_path, extra_file_paths=extra_file_paths)

        # Change path:
        yield eng.cd(project_path)
    except:
        
        # Remove extra files that are no longer needed:
        if clean:
            remove_extra_files(project_path=project_path, extra_file_paths=extra_file_paths)
        raise ValueError(project_path)
    else:
        eng.cd(old_path)
        
        # Remove extra files that are no longer needed:
        if clean:
            remove_extra_files(project_path=project_path, extra_file_paths=extra_file_paths)

In [169]:
extra_file_paths = [
    r'matlab/runner.m',
    r'matlab/TELIA-2012-01-25-2019-07-02.csv',
    r'matlab/ERIC_B-2015-01-04-2019-01-04.csv',
    r'matlab/HM_B-2015-10-01-2019-10-01.csv',
    r'matlab/VOLV_B-2012-07-03-2019-07-04.csv',
    
]

In [177]:
df['runnable']=False

In [178]:
df['complete']=None

In [250]:
incompletes = [
'Carl Edsbagge',
'Karl Egeland',
'Jonathan Ekdahl',
'Martin Engelsen',
'Love Engh',
'Vigg Eriksson',
'David Espedalen',
'Rasmus Fischer',
'Frida Gotenius',
'Alexander Gustafsson',
]
df.loc[incompletes,'complete']=False

completes = [
'Mohammed Ali El Masri',
'Benjamin Elm Jonsson',
'Daniel Eriksson',
'Fabian Fagerlind',
'Hilda Fahl',
'Sofia Friberg',
'Emelie Gillerstedt',
'May Venke Gylseth',
'Petrus Dahlqvist',
]
df.loc[completes,'complete']=True


In [247]:
import matlab.engine

df_summary = pd.DataFrame()


with run_matlab() as eng:
    
    for index, row in df.iterrows():
        
        print('%s : '%index, end = '')
        if not row['has_files']:
            print('Missing files')
            continue
            
        if row['runnable']:
            print('Runnable')
            continue  # Look at this one later...
        
        if row['complete'] is False:
            print('Incomplete')
            continue  # The student needs to fix this one
            
        if row['complete'] is True:
            print('Complete')
            continue  # I double checked this one and it looks good.
        
        with evaluate_path(row['dir_path'], eng=eng, extra_file_paths=extra_file_paths, clean=False) as path:
            r=eng.runner()
            df_transation = matlab_to_data_frame(keys = ['buy', 'sell', 'gain'], r=r)
            df_stock = matlab_to_data_frame(keys = ['high', 'low', 'close', 'FastK', 'K', 'SlowK'], r=r)
            
            s = df_stock.mean()
            s.name = index
            
        print('Runnable')
        df_summary=df_summary.append(s) 
        df.loc[index,'runnable']=True
           
            
        

Isak Dahlén : Runnable
Petrus Dahlqvist : Incomplete
Isac Danielsson : Runnable
Kirill Durkin : Runnable
Philip Dynevik : Runnable
Carl Edsbagge : Incomplete
Karl Egeland : Incomplete
Jonathan Ekdahl : Incomplete
Mohammed Ali El Masri : Complete
Benjamin Elm Jonsson : Complete
Johan Elnertz : Runnable
Martin Engelsen : Incomplete
Love Engh : Incomplete
Daniel Eriksson : Complete
Vigg Eriksson : Incomplete
Johan Erlandsson : Missing files
Jonathan Erneborg : Runnable
David Espedalen : Incomplete
Fabian Fagerlind : Complete
Hilda Fahl : Complete
Bekir Fazlija : Runnable
Rasmus Fischer : Incomplete
Sofia Friberg : Complete
Emelie Gillerstedt : Complete
Milton Göransson : Runnable
Anna Götberg : Runnable
Frida Gotenius : Incomplete
Elin Gryte : Runnable
Christoffer Gustafsson Blüme : Runnable
Alexander Gustafsson : Incomplete
May Venke Gylseth : Complete


In [248]:
mask = df['runnable']==True
show(df.loc[mask])

,family name,name,dir_path,id_string,has_files,runnable,complete
Isak Dahlén,Dahlén,Isak,path,dahlénisak,True,True,None
Isac Danielsson,Danielsson,Isac,path,danielssonisac,True,True,None
Kirill Durkin,Durkin,Kirill,path,durkinkirill,True,True,None
Philip Dynevik,Dynevik,Philip,path,dynevikphilip,True,True,None
Johan Elnertz,Elnertz,Johan,path,elnertzjohan,True,True,None
Jonathan Erneborg,Erneborg,Jonathan,path,erneborgjonathan,True,True,None
Bekir Fazlija,Fazlija,Bekir,path,fazlijabekir,True,True,None
Milton Göransson,Göransson,Milton,path,göranssonmilton,True,True,None
Anna Götberg,Götberg,Anna,path,götberganna,True,True,None
Elin Gryte,Gryte,Elin,path,gryteelin,True,True,None


In [249]:
mask = df['complete']==False
show(df.loc[mask])

,family name,name,dir_path,id_string,has_files,runnable,complete
Petrus Dahlqvist,Dahlqvist,Petrus,path,dahlqvistpetrus,True,False,False
Carl Edsbagge,Edsbagge,Carl,path,edsbaggecarl,True,False,False
Karl Egeland,Egeland,Karl,path,egelandkarl,True,False,False
Jonathan Ekdahl,Ekdahl,Jonathan,path,ekdahljonathan,True,False,False
Martin Engelsen,Engelsen,Martin,path,engelsenmartin,True,False,False
Love Engh,Engh,Love,path,enghlove,True,False,False
Vigg Eriksson,Eriksson,Vigg,path,erikssonvigg,True,False,False
David Espedalen,Espedalen,David,path,espedalendavid,True,False,False
Rasmus Fischer,Fischer,Rasmus,path,fischerrasmus,True,False,False
Frida Gotenius,Gotenius,Frida,path,goteniusfrida,True,False,False
